## List S3 Buckets and Create an S3 Bucket

In [3]:
import boto3
s3 = boto3.client('s3')
bucket_name = 'sheenutherider'

In [5]:
response = s3.list_buckets()   #displays the list of all the available buckets
response

{'ResponseMetadata': {'RequestId': 'GP8HW364J9G91E0N',
  'HostId': 'OoQ8QRDxlxEHGw8F/kaVCIBhUsxH+WLwn8MtcjgbVtoEiQ2MJtclyGaYBqALfcYonEcJzE5bcWI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'OoQ8QRDxlxEHGw8F/kaVCIBhUsxH+WLwn8MtcjgbVtoEiQ2MJtclyGaYBqALfcYonEcJzE5bcWI=',
   'x-amz-request-id': 'GP8HW364J9G91E0N',
   'date': 'Thu, 13 Feb 2025 16:53:01 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [],
 'Owner': {'DisplayName': 'shrianshsingh16',
  'ID': 'b454deb4f76f8ee93d5fed1d38338125c4ee1bdecaf411e5560a36ae6babca7e'}}

In [6]:
def create_bucket(bucket_name):
    s3.create_bucket(Bucket=bucket_name)
    print("Bucket is created")

create_bucket(bucket_name)

Bucket is created


## Upload a Single File to S3

In [7]:
import os

def upload_file(file_path, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_path)

    s3.upload_file(file_path, bucket_name, object_name)


In [8]:
upload_file('data/index.html')

In [9]:
#we can even upload the file with the new name
upload_file('data/index.html', 'somefile.html')

## List all objects in a bucket

- list all objects in a bucket

In [10]:
def list_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)
    for obj in response['Contents']:
        print(obj['Key'])

list_objects()

index.html
somefile.html


## Download S3 File to Local System

In [11]:
def download_file(object_name, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))

    s3.download_file(bucket_name, object_name, file_path)


download_file('index.html', 'data_download/index.html')

## Upload a Folder to S3

In [12]:
def upload_directory(directory_path, s3_prefix):
    for root, dirs, files in os.walk('data'):
        for file in files:
            file_path = os.path.join(root, file).replace("\\", "/")
            relpath = os.path.relpath(file_path, directory_path)
            s3_key = os.path.join(s3_prefix, relpath).replace("\\", "/")
            
            s3.upload_file(file_path, bucket_name, s3_key)

upload_directory('data', 's3_data')

## Download S3 Folder to Local System

In [13]:
local_path = 's3_download_1'
s3_prefix = 's3_data'

def download_dir(local_path, s3_prefix):
    os.makedirs(local_path, exist_ok=True)
    paginator = s3.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
        if 'Contents' in result:
            for key in result['Contents']:
                s3_key = key['Key']

                local_file = os.path.join(local_path, os.path.relpath(s3_key, s3_prefix))
                # os.makedirs(os.path.dirname(local_file), exist_ok=True)

                s3.download_file(bucket_name, s3_key, local_file)



download_dir(local_path, s3_prefix)

## Delete All Files in an S3 Bucket

In [14]:
def delete_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:

        for obj in response['Contents']:
            s3.delete_object(Bucket=bucket_name, Key=obj['Key'])



delete_objects()

In [15]:
s3.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'YBQ9FEM52EFTN0QY',
  'HostId': '9YzraGvtWJROiwNAjjwR2WsAsW5gK8K2C5Y5FhnPuaN6blu0oL8uPYsQefVZ6oU9C5ZFyWZU6T4=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '9YzraGvtWJROiwNAjjwR2WsAsW5gK8K2C5Y5FhnPuaN6blu0oL8uPYsQefVZ6oU9C5ZFyWZU6T4=',
   'x-amz-request-id': 'YBQ9FEM52EFTN0QY',
   'date': 'Thu, 13 Feb 2025 17:32:47 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}